# Gameplan

To get different particle numbers at the same couplings, I will increment different solutions simultaneously! Woah that's crazy.

In [1]:
from solve_rg_eqs import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


l = 10
L = 2*l
print('Ns:')
Ns = np.arange(2, 4*l, 2)
print(Ns)
k = np.arange(1, 2*l+1, 2)*0.5*np.pi/l
Gc = 1./np.sum(k)
dg = 0.01/L
g0 = .1*dg
imk = .1*g0
imv = .1*g0

gs = np.arange(g0, G_to_g(.2*Gc, k), dg)
qs = np.arange(1/gs[-1], 1/G_to_g(1.2*Gc, k), -10*dg)
print(len(gs))
print(len(qs))

Ns:
[ 2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38]
32
13386


In [2]:
def bootstrap_g0_multi(L, g0, kc, imscale_v):
    vs = np.zeros(L)

    Ns = np.arange(2, 4*L, 2)
    Ne = Ns[-1]//2
    sols = [None for N in Ns]
    for i, N in enumerate(Ns):
        log('')
        log('Now using {} fermions'.format(N))
        log('')
        n = N//2
        dims = (L, n, n)
        print('Dimensions')
        print(dims)
        # Solving for 2n fermions using extrapolation from previous solution
        if n <= 2:
            force_gs=False
            noise_factors=None
            vars = g0_guess(L, n, n, kc, g0, imscale=imscale_v)
        else:
            # The previous solution matches to roughly the accuracy of the solution
            # for the shared variables
            noise_factors = 10*er*np.ones(len(vars))
            # But we will still need to try random stuff for the 4 new variables
            noise_factors[n-2:n] = 1
            noise_factors[2*n-2:2*n] = 1
            noise_factors[3*n-2:3*n] = 1
            noise_factors[4*n-2:4*n] = 1
        sol = find_root_multithread(vars, kc, g0, dims, imscale_v,
                                    max_steps=MAX_STEPS_1,
                                    use_k_guess=False,
                                    noise_factors=noise_factors,
                                    force_gs=force_gs,
                                    factor=1.4)
        print(vars)
        vars = sol.x
        er = max(abs(rgEqs(vars, kc, g0, dims)))
        log('Error with {} fermions: {}'.format(2*n, er))
        sols[i] = sol
        
        if n%2 == 1:
            # setting up for N divisible by 4 next step
            if n > 1:
                vars = sols[i-1].x # Better to start from last similar case
            n -= 1
            incr = 2
        else:
            incr = 1
        
        if n >= 1:
            es, ws = unpack_vars(vars, n, n)
            vars_guess = g0_guess(L, n+incr, n+incr, kc, g0, imscale=imscale_v)
            esg, wsg = unpack_vars(vars_guess, n+incr, n+incr)
            es = np.append(es, esg[-incr:])
            ws = np.append(ws, wsg[-incr:])
            vars = pack_vars(es, ws)        

    return sols, Ns


def solve_job(prev_vars, kc, g, dims, prev_g):
    sol = root(rgEqs, prev_vars, args=(kc, g, dims), method='lm',
               jac=rg_jac)
    er = np.max(np.abs(sol.fun))
    if er > 10**-6:
        gts = np.linspace(prev_g, g, 10)
        for gt in gts:
            sol = root(rgEqs, prev_vars, args=(kc, gt, dims), method='lm',
                       jac=rg_jac)
            prev_vars = sol.x
    return sol


def solve_g_all(l, Ns, g, kc, prev_g, prev_sols):
    dims = [(l, N//2, N//2) for N in Ns]
    with concurrent.futures.ProcessPoolExecutor(max_workers=CPUS) as executor:
        future_results = [executor.submit(solve_job, ps.x, kc, g, dims[Ni], prev_g) for Ni, ps in enumerate(prev_sols)]
        concurrent.futures.wait(future_results)
        for res in future_results:
            try:
                yield res.result()
            except Exception as e:
                print(e)

In [3]:
kim = imk*(-1)**np.arange(1,l+1)
print(kim)
kc = np.concatenate((k, kim))
skip = 10

Ns = np.arange(2, 4*l, 2)
print(Ns)
prev_sols = [None for N in Ns]
energies = np.zeros((len(gs//skip), len(Ns)))
ers = np.zeros((len(gs//skip), len(Ns)))
# varss = []
# gss = []
for gi, g in enumerate(gs):
    print('')
    if g == gs[0]:
        prev_sols, Ns = bootstrap_g0_multi(l, g, kc, imscale_v=imv)
    else:
        print('g = {}'.format(g))
        g_prev = gs[gi-1]
        for i, s in enumerate(solve_g_all(l, Ns, g, kc, g_prev, prev_sols)):
            prev_sols[i] = s
    for Ni, ps in enumerate(prev_sols):
        N = Ns[Ni]
        print('Error for N = {}'.format(N))
        print(np.max(np.abs(ps.fun)))

[-5.e-06  5.e-06 -5.e-06  5.e-06 -5.e-06  5.e-06 -5.e-06  5.e-06 -5.e-06
  5.e-06]
[ 2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38]


Now using 2 fermions

Dimensions
(10, 1, 1)
[ 1.57078399e-01 -4.64996073e-06  0.00000000e+00  0.00000000e+00]
Error with 2 fermions: 1.0111580195371062e-11

Now using 4 fermions

Dimensions
(10, 2, 2)
Bad initial guess. Trying with noise.
g = 5e-05, er = 0.0004351721178740263
1th try at g = 5e-05
Smallest error from last set: 0.0004351721178740263
Noise ranges from -4.854177876278173e-06 to 5.607088661094938e-06
3th try at g = 5e-05
Smallest error from last set: 0.0004452354021370411
Noise ranges from -3.1142533868990256e-06 to 8.486378238353733e-06
[ 1.57078399e-01  1.57078399e-01 -4.64996073e-06 -5.34996073e-06
  1.57079221e-01  1.57079221e-01 -2.09999869e-05  1.10000131e-05]
Error with 4 fermions: 3.7584504752885554e-11

Now using 6 fermions

Dimensions
(10, 3, 3)
[ 1.57078850e-01  1.57076714e-01  4.71227795e-01 -4.99995020e-06
 -4.99981415e

KeyboardInterrupt: 

In [ ]:
final_energies = np.zeros(len(Ns))
for Ni, N in enumerate(Ns):
    print('Removing imaginary parts at N = {}'.format(N))
    dims = (l, N//2, N//2)
    vars_r, er_r = increment_im_k(prev_sols[Ni].x, dims, g, k, kc[l:], force_gs=False, steps=50, max_steps=5)
    varss = np.transpose(np.array([vars_r]))
    energy, _ = calculate_energies(dims, [g], k, varss)
    final_energies[Ni] = energy[0]

In [ ]:
plt.plot(Ns, final_energies)

In [ ]:
print(np.shape(gs))
print(np.shape(ers))
Gs = g_to_G(gs, k)
for Ni, N in enumerate(Ns):
    if np.max(ers[:, Ni]) < 10**-6:
        plt.plot(energies[:5, Ni])